Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

## Library imports


In [21]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List
import time
import pandas as pd
import json

# Seed
np.random.seed(42)

# Settings matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Nearest Neighbor Initialization (Greedy)

In [ ]:
def calculate_cost(tour: np.ndarray, distance_matrix: np.ndarray) -> float:

    cost = 0
    for i in range(len(tour)):
        cost += distance_matrix[tour[i], tour[(i + 1) % len(tour)]]
    return cost

def nearest_neighbor(distance_matrix: np.ndarray, start_city: int = 0) -> np.ndarray:

    n = len(distance_matrix)
    unvisited = set(range(n)) 
    tour = [start_city]
    unvisited.remove(start_city)
    
    current = start_city
    
    while unvisited:
        nearest = min(unvisited, key=lambda city: distance_matrix[current, city])
        tour.append(nearest)
        unvisited.remove(nearest)
        current = nearest
    
    return np.array(tour)

def initialize_population(distance_matrix: np.ndarray, pop_size: int) -> List[np.ndarray]:

    n_cities = distance_matrix.shape[0]
    population = []
    
    for start in range(n_cities):
        tour = nearest_neighbor(distance_matrix, start)
        population.append(tour)
    
    while len(population) < pop_size:
        population.append(np.random.permutation(n_cities))
    
    return population[:pop_size]

## GA Components

In [ ]:
def tournament_selection(population: List[np.ndarray], 
                         fitness: np.ndarray, 
                         tournament_size: int = 3) -> np.ndarray:
    
    candidates = np.random.choice(len(population), tournament_size, replace=False)
    
    best_idx = candidates[np.argmin(fitness[candidates])]
    
    return population[best_idx].copy()

def order_crossover(parent1: np.ndarray, parent2: np.ndarray) -> np.ndarray:

    size = len(parent1)
    
    start, end = sorted(np.random.choice(size, 2, replace=False))
    
    child = np.full(size, -1, dtype=int)
    
    child[start:end] = parent1[start:end]
    
    current_pos = end
    for city in parent2:
        if city not in child:
            if current_pos >= size:
                current_pos = 0
            child[current_pos] = city
            current_pos += 1
    
    return child

def swap_mutation(tour: np.ndarray, mutation_rate: float) -> np.ndarray:

    tour = tour.copy()
    
    if np.random.random() < mutation_rate:
        i, j = np.random.choice(len(tour), 2, replace=False)
        tour[i], tour[j] = tour[j], tour[i]
    
    return tour

def inversion_mutation(tour: np.ndarray, mutation_rate: float) -> np.ndarray:

    tour = tour.copy()
    if np.random.random() < mutation_rate:
        i, j = sorted(np.random.choice(len(tour), 2, replace=False))
        tour[i:j+1] = tour[i:j+1][::-1]
    return tour

def insertion_mutation(tour: np.ndarray, mutation_rate: float) -> np.ndarray:

    tour = tour.copy()
    if np.random.random() < mutation_rate:
        remove_idx = np.random.randint(len(tour))
        city = tour[remove_idx]
        
        tour = np.delete(tour, remove_idx)
        
        insert_idx = np.random.randint(len(tour) + 1)
        
        tour = np.insert(tour, insert_idx, city)
    return tour

def adaptive_mutation(tour: np.ndarray, mutation_rate: float, 
                      generation: int, max_generations: int) -> np.ndarray:
    
    if np.random.random() > mutation_rate:
        return tour
    
    phase = generation / max_generations if max_generations > 0 else 0
    
    rand = np.random.random()
    
    if phase < 0.3:
        if rand < 0.5:
            return inversion_mutation(tour, 1.0)
        else:
            return insertion_mutation(tour, 1.0)
    
    elif phase < 0.7:
        if rand < 0.5:
            return swap_mutation(tour, 1.0)
        elif rand < 0.75:
            return inversion_mutation(tour, 1.0)
        else:
            return insertion_mutation(tour, 1.0)
    
    else:
        if rand < 0.7:
            return swap_mutation(tour, 1.0)
        else:
            return inversion_mutation(tour, 1.0)

## Local Search


In [ ]:
def two_opt(tour: np.ndarray, distance_matrix: np.ndarray, max_iter: int = 100) -> np.ndarray:

    best_tour = tour.copy()
    best_cost = calculate_cost(best_tour, distance_matrix)
    improved = True
    iterations = 0
    
    while improved and iterations < max_iter:
        improved = False
        iterations += 1
        
        for i in range(len(tour) - 1):
            for j in range(i + 2, len(tour)):
                new_tour = best_tour.copy()
                new_tour[i:j+1] = new_tour[i:j+1][::-1]
                new_cost = calculate_cost(new_tour, distance_matrix)
                
                if new_cost < best_cost:
                    best_tour = new_tour
                    best_cost = new_cost
                    improved = True
                    break
            
            if improved:
                break
    
    return best_tour

## GA algorithm

In [ ]:
from typing import Tuple

class GeneticAlgorithmTSP:
    
    def __init__(self, 
                 distance_matrix: np.ndarray,
                 pop_size: int = 100,
                 elite_size: int = 10,
                 mutation_rate: float = 0.2,
                 tournament_size: int = 3,
                 use_2opt: bool = True,
                 two_opt_freq: int = 10):
        
        self.distance_matrix = distance_matrix
        self.n_cities = distance_matrix.shape[0]
        self.pop_size = pop_size
        self.elite_size = elite_size
        self.mutation_rate = mutation_rate
        self.tournament_size = tournament_size
        self.use_2opt = use_2opt
        self.two_opt_freq = two_opt_freq
        
        self.best_costs = []
        self.avg_costs = []
        self.best_tour = None
        self.best_cost = float('inf')
    
    def evolve(self, max_generations: int = 500, verbose: bool = True) -> Tuple[np.ndarray, float]:
        population = initialize_population(self.distance_matrix, self.pop_size)
        
        for generation in range(max_generations):
            fitness = np.array([calculate_cost(tour, self.distance_matrix) for tour in population])
            
            gen_best_idx = np.argmin(fitness)
            if fitness[gen_best_idx] < self.best_cost:
                self.best_cost = fitness[gen_best_idx]
                self.best_tour = population[gen_best_idx].copy()
            
            self.best_costs.append(self.best_cost)
            self.avg_costs.append(np.mean(fitness))
            
            if verbose and (generation % 10 == 0 or generation == max_generations - 1):
                print(f"Gen {generation:4d}/{max_generations} | "
                      f"Best: {self.best_cost:8.2f} | "
                      f"Avg: {np.mean(fitness):8.2f}")
            
            new_population = []
            
            elite_indices = np.argsort(fitness)[:self.elite_size]
            for idx in elite_indices:
                new_population.append(population[idx].copy())
            
            while len(new_population) < self.pop_size:
                parent1 = tournament_selection(population, fitness, self.tournament_size)
                parent2 = tournament_selection(population, fitness, self.tournament_size)
                child = order_crossover(parent1, parent2)
                child = adaptive_mutation(child, self.mutation_rate, generation, max_generations)
                new_population.append(child)
            
            if self.use_2opt and generation % self.two_opt_freq == 0:
                for i in range(min(self.elite_size, len(new_population))):
                    new_population[i] = two_opt(new_population[i], self.distance_matrix, max_iter=50)
            
            population = new_population
        
        return self.best_tour, self.best_cost
    
    def plot_convergence(self):
        plt.figure(figsize=(12, 5))
        plt.plot(self.best_costs, label='Best Cost', linewidth=2, color='green')
        plt.plot(self.avg_costs, label='Average Cost', alpha=0.7, color='blue')
        plt.xlabel('Generation')
        plt.ylabel('Cost')
        plt.title('GA Convergence')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## Solve function


In [ ]:
def solve_problem(problem_file: str) -> dict:
    problem = np.load(f'lab2/{problem_file}')
    n_cities = problem.shape[0]
    problem_name = problem_file.replace('.npy', '')
    
    if n_cities <= 50:
        generations, pop_size, two_opt_freq = 300, 100, 10
    elif n_cities <= 200:
        generations, pop_size, two_opt_freq = 500, 100, 20
    else:
        generations, pop_size, two_opt_freq = 1000, 150, 50
    
    print(f"\n{'='*70}")
    print(f"Solving: {problem_name} ({n_cities} cities)")
    print(f"Parameters: {generations} gen, pop={pop_size}, 2opt every {two_opt_freq}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    ga = GeneticAlgorithmTSP(
        distance_matrix=problem,
        pop_size=pop_size,
        elite_size=max(10, pop_size // 10),
        mutation_rate=0.2,
        use_2opt=True,
        two_opt_freq=two_opt_freq
    )
    
    best_tour, best_cost = ga.evolve(max_generations=generations, verbose=True)
    elapsed = time.time() - start_time
    
    print(f"\n Solved in {elapsed:.1f}s | Best cost: {best_cost:.2f}")
    
    return {
        'problem': problem_name,
        'n_cities': n_cities,
        'best_cost': best_cost,
        'best_tour': best_tour.tolist(),
        'time': elapsed,
        'history': ga.best_costs
    }

## Problem g

In [29]:
problem_g = [
    'problem_g_10.npy', 'problem_g_20.npy', 'problem_g_50.npy', 
    
]

results_g = []
for problem_file in problem_g:
    result = solve_problem(problem_file)
    results_g.append(result)

df = pd.DataFrame([{
    'Problem': r['problem'],
    'Cities': r['n_cities'],
    'Best Cost': f"{r['best_cost']:.2f}",
    'Time (s)': f"{r['time']:.1f}"
} for r in results_g])

print("\n" + "="*70)
print("🏆 FINAL RESULTS")
print("="*70)
print(df.to_string(index=False))




Solving: problem_g_10 (10 cities)
Parameters: 300 gen, pop=100, 2opt every 10
Gen    0/300 | Best:  1497.66 | Avg:  2966.32
Gen   10/300 | Best:  1497.66 | Avg:  2023.31
Gen   20/300 | Best:  1497.66 | Avg:  1980.54
Gen   30/300 | Best:  1497.66 | Avg:  1954.33
Gen   40/300 | Best:  1497.66 | Avg:  1992.79
Gen   50/300 | Best:  1497.66 | Avg:  2020.89
Gen   60/300 | Best:  1497.66 | Avg:  2069.28
Gen   70/300 | Best:  1497.66 | Avg:  1913.77
Gen   80/300 | Best:  1497.66 | Avg:  1989.06
Gen   90/300 | Best:  1497.66 | Avg:  2023.89
Gen  100/300 | Best:  1497.66 | Avg:  2050.81
Gen  110/300 | Best:  1497.66 | Avg:  2088.18
Gen  120/300 | Best:  1497.66 | Avg:  1980.19
Gen  130/300 | Best:  1497.66 | Avg:  1954.97
Gen  140/300 | Best:  1497.66 | Avg:  1966.83
Gen  150/300 | Best:  1497.66 | Avg:  2022.50
Gen  160/300 | Best:  1497.66 | Avg:  2007.05
Gen  170/300 | Best:  1497.66 | Avg:  2090.34
Gen  180/300 | Best:  1497.66 | Avg:  2062.95
Gen  190/300 | Best:  1497.66 | Avg:  1990.51
G

## Problem r1

In [ ]:
problem_r1 = [
    'problem_r1_10.npy', 'problem_r1_20.npy', 'problem_r1_50.npy',
    'problem_r1_100.npy', 'problem_r1_200.npy', 'problem_r1_500.npy', 'problem_r1_1000.npy',
]

results_r1 = []
for problem_file in problem_r1:
    result = solve_problem(problem_file)
    results_r1.append(result)

df = pd.DataFrame([{
    'Problem': r['problem'],
    'Cities': r['n_cities'],
    'Best Cost': f"{r['best_cost']:.2f}",
    'Time (s)': f"{r['time']:.1f}"
} for r in results_r1])

print("\n" + "="*70)
print("🏆 FINAL RESULTS")
print("="*70)
print(df.to_string(index=False))

## Problem r2

In [ ]:
problem_r2 = [
    'problem_r1_10.npy', 'problem_r1_20.npy', 'problem_r1_50.npy',
    'problem_r1_100.npy', 'problem_r1_200.npy', 'problem_r1_500.npy', 'problem_r1_1000.npy',
]

results_r2 = []
for problem_file in problem_r2:
    result = solve_problem(problem_file)
    results_r2.append(result)

df = pd.DataFrame([{
    'Problem': r['problem'],
    'Cities': r['n_cities'],
    'Best Cost': f"{r['best_cost']:.2f}",
    'Time (s)': f"{r['time']:.1f}"
} for r in results_r2])

print("\n" + "="*70)
print("🏆 FINAL RESULTS")
print("="*70)
print(df.to_string(index=False))